# Regression, Part B

### Baseline

In [42]:
# Import
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
import torch
import importlib_resources

from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from scipy import stats

from dtuimldmtools import draw_neural_net, train_neural_net, rlr_validate

In [39]:
# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('./wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# One of K encoding
# y = similarity.one_of_k(y)

# Ensure y values are within the range of the identity matrix's size
num_classes = int(np.max(y)) + 1
y2 = np.eye(num_classes)[y.astype(int)]

#removing first column of y
y2 = y2[:,1:]

#Appending y to X
X = np.append(X,y2,axis=1)

#Extracting the first column of X
y = X[:,0]



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

#Removing the first column of X
X = np.delete(X,0,axis=1)
# Removing alcohol from attributenames
attributeNames = attributeNames[1:]


N, M = X.shape

# Add offset attribute
X = np.concatenate((np.ones((X.shape[0], 1)), X), 1)
attributeNames = ["Offset"] + attributeNames + ["Class 1", "Class 2", "Class 3"]
M = M + 1


In [41]:
# Step 2: Set up K-Fold cross-validation (using outer loop)
K = 10
CV = model_selection.KFold(n_splits=K, shuffle=True)

# Initialize arrays to store errors for each fold
baseline_errors_train = np.empty(K)
baseline_errors_test = np.empty(K)

k = 0
for train_index, test_index in CV.split(X):
    print(f"Computing fold {k + 1}/{K}...")

    # Extract training and test sets for the current fold
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]

    # Step 3: Baseline model - predict the mean of y_train
    mean_y_train = np.mean(y_train)

    # Predict the mean for both training and test sets
    y_pred_train = np.full(len(y_train), mean_y_train)
    y_pred_test = np.full(len(y_test), mean_y_train)

    # Step 4: Calculate Mean Squared Error (MSE) for training and test sets
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Store the errors
    baseline_errors_train[k] = mse_train
    baseline_errors_test[k] = mse_test

    print(f"     Fold {k + 1}/{K} - MSE test: {mse_test:.4f}")

    k += 1

# Step 5: Print the results
print("\nBaseline Model Results:")
print(f"Average training MSE: {np.mean(baseline_errors_train):.4f}")
print(f"Average test MSE: {np.mean(baseline_errors_test):.4f}")

Computing fold 1/10...
     Fold 1/10 - MSE test: 1.5580
Computing fold 2/10...
     Fold 2/10 - MSE test: 0.7153
Computing fold 3/10...
     Fold 3/10 - MSE test: 0.9252
Computing fold 4/10...
     Fold 4/10 - MSE test: 0.7953
Computing fold 5/10...
     Fold 5/10 - MSE test: 1.0276
Computing fold 6/10...
     Fold 6/10 - MSE test: 0.7982
Computing fold 7/10...
     Fold 7/10 - MSE test: 0.7760
Computing fold 8/10...
     Fold 8/10 - MSE test: 0.9052
Computing fold 9/10...
     Fold 9/10 - MSE test: 1.2166
Computing fold 10/10...
     Fold 10/10 - MSE test: 1.3735

Baseline Model Results:
Average training MSE: 0.9997
Average test MSE: 1.0091


### Fit ANN model - Do not use, friends

In [51]:
# ANN
# Using script from: ex8_3_1 with modifications

# Define K-Fold cross-validation
K = 10
CV = model_selection.KFold(n_splits=K, shuffle=True)

# Define the model structure for regression
n_hidden_units = 5                            # Adjust as needed
model = lambda: torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], n_hidden_units),    # Input layer to hidden layer
    torch.nn.ReLU(),                                # Activation function
    torch.nn.Linear(n_hidden_units, 1)              # Output layer with one output (regression)
)

# Use MSE loss function for regression
loss_fn = torch.nn.MSELoss()

# Initialize variables to store errors
Error_train = np.empty(K)
Error_test = np.empty(K)

k = 0
for train_index, test_index in CV.split(X):
    print(f"Computing CV fold {k + 1}/{K}...")

    # Extract training and test sets
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]

    # Convert data to torch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float).view(-1, 1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float).view(-1, 1)

    # Train the neural network
    net, _, _ = train_neural_net(
        model,
        loss_fn,
        X=X_train_tensor,
        y=y_train_tensor,
        n_replicates=3,
        max_iter = 1000
    )

    # Make predictions on the training and test sets
    y_train_pred = net(X_train_tensor).detach().numpy().flatten()
    y_test_pred = net(X_test_tensor).detach().numpy().flatten()

    # Calculate Mean Squared Error (MSE)
    Error_train[k] = mean_squared_error(y_train, y_train_pred)
    Error_test[k] = mean_squared_error(y_test, y_test_pred)

    print(f"     Fold {k + 1}/{K} - MSE test: {Error_test[k]:.4f}")

    k += 1

# Print the results
print("\nANN Model Results (Regression):")
print(f"Average training MSE: {np.mean(Error_train):.4f}")
print(f"Average test MSE: {np.mean(Error_test):.4f}")


Computing CV fold 1/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.06142568	0.00070700905
		Final loss:
		1000	0.06142568	0.00070700905

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.5073065	0.0013767574
		Final loss:
		1000	0.5073065	0.0013767574

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.15765914	0.0009507672
		Final loss:
		1000	0.15765914	0.0009507672
     Fold 1/10 - MSE test: 0.0755
Computing CV fold 2/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.11802695	0.0011479544
		Final loss:
		1000	0.11802695	0.0011479544

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.05525775	0.0009269947
		Final loss:
		1000	0.05525775	0.0009269947

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.06734678	0.0025449684
		Final loss:
		1000	0.06734678	0.0025449684
     Fold 2/10 - MSE test: 0.1373
Computing CV fold 3/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.09687764	0.0014322635
		Final loss:
		1000	0.09687764	0.0014322635

	Replicate: 2/3
		Iter	Loss			Rel.